# Section 0: Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base
import pymongo
from sqlalchemy.orm import Session
%matplotlib inline

# Import API key
from config import gkey

# Section 1: Read Datasets

In [2]:
banzuke = pd.read_csv('banzuke.csv')
banzuke.head()

,basho,id,rank,rikishi,heya,shusshin,birth_date,height,weight,prev,prev_w,prev_l
0,1983.01,1354,Y1e,Chiyonofuji,Kokonoe,Hokkaido,1955-06-01,182.0,116.0,Y1e,14.0,1.0
1,1983.01,4080,Y1w,Kitanoumi,Mihogaseki,Hokkaido,1953-05-16,179.0,165.0,Y2eHD,9.0,3.0
2,1983.01,4095,Y2eHD,Wakanohana,Futagoyama,Aomori,1953-04-03,186.0,133.0,Y1w,0.0,0.0
3,1983.01,4104,O1e,Takanosato,Futagoyama,Aomori,1952-09-29,181.0,144.0,O1e,10.0,5.0
4,1983.01,4112,O1w,Kotokaze,Sadogatake,Mie,1957-04-26,183.0,163.0,O1w,10.0,5.0


In [3]:
results = pd.read_csv('results.csv')
results.head()

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),0,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),1
1,1983.01,1,4306,Ms1e,Ofuji,1-0 (6-1),1,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),0
2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),1,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),0
3,1983.01,1,4323,J13e,Shiraiwa,0-1 (3-12),0,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),1
4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),0,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),1


In [4]:
banzuke[banzuke['basho']==2021.03]['rikishi'].value_counts()

Tanahashi       1
Wakatakakage    1
Tabara          1
Suonada         1
Tochikodai      1
               ..
Ichiyamamoto    1
Kototsubasa     1
Ofukasawa       1
Ishiura         1
Hakuyo          1
Name: rikishi, Length: 654, dtype: int64

In [5]:
banzuke[(banzuke['basho']==2021.03) & (banzuke['rikishi']=='Terunofuji')]

,basho,id,rank,rikishi,heya,shusshin,birth_date,height,weight,prev,prev_w,prev_l
167785,2021.03,11927,S1e,Terunofuji,Isegahama,Mongolia,1991-11-29,192.0,158.5,S1e,11.0,4.0


In [7]:
results[(results['basho']==2021.03) & (results['rikishi1_shikona']=='Terunofuji')]

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
218856,2021.03,1,11927,S1e,Terunofuji,1-0 (12-3),1,shitatenage,12239,M2e,Hokutofuji,0-1 (9-6),0
218896,2021.03,2,11927,S1e,Terunofuji,2-0 (12-3),1,oshidashi,12370,M2w,Wakatakakage,0-2 (10-5),0
218940,2021.03,3,11927,S1e,Terunofuji,3-0 (12-3),1,yoritaoshi,11985,K2w,Daieisho,0-3 (8-7),0
218980,2021.03,4,11927,S1e,Terunofuji,4-0 (12-3),1,kotenage,11946,M3e,Meisei,2-2 (10-5),0
219022,2021.03,5,11927,S1e,Terunofuji,4-1 (12-3),0,oshidashi,12043,M1w,Onosho,2-3 (4-11),1
219060,2021.03,6,11927,S1e,Terunofuji,5-1 (12-3),1,tsuridashi,12231,M4e,Kiribayama,2-4 (7-8),0
219102,2021.03,7,11927,S1e,Terunofuji,6-1 (12-3),1,yorikiri,12210,K1w,Mitakeumi,3-4 (8-7),0
219141,2021.03,8,11927,S1e,Terunofuji,6-2 (12-3),0,yorikiri,6480,K1e,Takayasu,7-1 (10-5),1
219183,2021.03,9,11927,S1e,Terunofuji,7-2 (12-3),1,kimedashi,11784,M4w,Myogiryu,5-4 (7-8),0
219222,2021.03,10,11927,S1e,Terunofuji,7-3 (12-3),0,tsukiotoshi,12024,M3w,Shimanoumi,3-7 (4-11),1


# Section 2: Limit Data to Focus on Top Division Only

In [8]:
topdiv = banzuke[(banzuke['rank'].str.contains('Y\d'))|(banzuke['rank'].str.contains('M\d'))|(banzuke['rank'].str.contains('K\d'))|(banzuke['rank'].str.contains('S\d'))|(banzuke['rank'].str.startswith('O'))]

In [9]:
topdiv.head()

,basho,id,rank,rikishi,heya,shusshin,birth_date,height,weight,prev,prev_w,prev_l
0,1983.01,1354,Y1e,Chiyonofuji,Kokonoe,Hokkaido,1955-06-01,182.0,116.0,Y1e,14.0,1.0
1,1983.01,4080,Y1w,Kitanoumi,Mihogaseki,Hokkaido,1953-05-16,179.0,165.0,Y2eHD,9.0,3.0
2,1983.01,4095,Y2eHD,Wakanohana,Futagoyama,Aomori,1953-04-03,186.0,133.0,Y1w,0.0,0.0
3,1983.01,4104,O1e,Takanosato,Futagoyama,Aomori,1952-09-29,181.0,144.0,O1e,10.0,5.0
4,1983.01,4112,O1w,Kotokaze,Sadogatake,Mie,1957-04-26,183.0,163.0,O1w,10.0,5.0


In [10]:
#Get unique fighters in Top Division
fighters = topdiv.copy()
fighters.drop_duplicates(subset='id',keep='last', inplace=True)

In [11]:
fighters[fighters['rikishi']=='Terunofuji']

,basho,id,rank,rikishi,heya,shusshin,birth_date,height,weight,prev,prev_w,prev_l
167785,2021.03,11927,S1e,Terunofuji,Isegahama,Mongolia,1991-11-29,192.0,158.5,S1e,11.0,4.0


In [12]:
#Drop not needed columns in fighters dataset
fighters.drop(['basho','prev','prev_w','prev_l'],axis=1,inplace=True)

In [13]:
fighters.head()

,id,rank,rikishi,heya,shusshin,birth_date,height,weight
2,4095,Y2eHD,Wakanohana,Futagoyama,Aomori,1953-04-03,186.0,133.0
36,4122,M14e,Kurosegawa,Isegahama,Tokyo,1951-05-13,184.0,130.0
37,4103,M14w,Banryuyama,Mihogaseki,Hyogo,1951-05-04,177.5,135.0
4492,4050,M8e,Takamiyama,Takasago,U.S.A.,1944-06-16,192.0,205.0
4497,4077,M10w,Fujizakura,Takasago,Yamanashi,1948-02-09,180.0,139.0


In [14]:
fighters.reset_index(drop=True, inplace=True)

In [15]:
#Save to CSV
fighters.to_csv('fighters.csv',index=False)

# Section 3: API Calls

In [16]:
#Make copy of fighters dataset to Add stable info on Lat and Lng
stables = fighters.copy()

In [17]:
#Create needed columns to store data for each stable
stables['lat'] =''
stables['lng'] =''

#Perform API call to get information for each stable
base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'


#Parameter for the API call
params = {'key':gkey}

#Iterate through the list of stables to retrieve their information
for i, row in stables.iterrows():
    
    params['address'] = row['heya']+ ' Japan'

    response = requests.get(base_url,params=params).json()
    
    
    
    try:
        stables.loc[i,'lat']= response['results'][0]['geometry']['location']['lat']
        stables.loc[i,'lng']= response['results'][0]['geometry']['location']['lng']
    except:
        #Assign NaN to Cities not found, for misisng values to be filled with closest location
        stables.loc[i,['lat','lng']]= np.nan

# Section 4: Clean Datasets

In [19]:
stables.head()

,id,rank,rikishi,heya,shusshin,birth_date,height,weight,lat,lng
0,4095,Y2eHD,Wakanohana,Futagoyama,Aomori,1953-04-03,186.0,133.0,43.0431,141.316
1,4122,M14e,Kurosegawa,Isegahama,Tokyo,1951-05-13,184.0,130.0,32.4138,131.65
2,4103,M14w,Banryuyama,Mihogaseki,Hyogo,1951-05-04,177.5,135.0,35.4958,136.918
3,4050,M8e,Takamiyama,Takasago,U.S.A.,1944-06-16,192.0,205.0,34.7661,134.791
4,4077,M10w,Fujizakura,Takasago,Yamanashi,1948-02-09,180.0,139.0,34.7661,134.791


In [20]:
stables['lat'] = stables['lat'].astype('float64')

In [21]:
stables['lng'] = stables['lng'].astype('float64')

In [22]:
stables.rename(columns={'rikishi':'fighter', 'heya':'dojo','shusshin':'district'}, inplace=True)

In [23]:
stables.reset_index(drop=True, inplace=True)

In [24]:
stables.head()

,id,rank,fighter,dojo,district,birth_date,height,weight,lat,lng
0,4095,Y2eHD,Wakanohana,Futagoyama,Aomori,1953-04-03,186.0,133.0,43.043111,141.316019
1,4122,M14e,Kurosegawa,Isegahama,Tokyo,1951-05-13,184.0,130.0,32.413809,131.649506
2,4103,M14w,Banryuyama,Mihogaseki,Hyogo,1951-05-04,177.5,135.0,35.495838,136.918181
3,4050,M8e,Takamiyama,Takasago,U.S.A.,1944-06-16,192.0,205.0,34.766101,134.790601
4,4077,M10w,Fujizakura,Takasago,Yamanashi,1948-02-09,180.0,139.0,34.766101,134.790601


In [25]:
stables.to_csv('Stables.csv',index=False)

In [26]:
results['basho'] = results['basho'].astype('str')

In [27]:
results.rename(columns={'basho':'tournament', 'rikishi1_id':'fighter1_id','rikishi1_rank':'fighter1_rank', \
                       'rikishi1_shikona':'fighter1_name', 'rikishi1_result':'fighter1_result', 'rikishi1_win':'fighter1_win', \
                       'kimarite':'finishing_move', 'rikishi2_id':'fighter2_id','rikishi2_rank':'fighter2_rank', \
                       'rikishi2_shikona':'fighter2_name', 'rikishi2_result':'fighter2_result', 'rikishi2_win':'fighter2_win'}, inplace=True)

In [28]:
results.head()

,tournament,day,fighter1_id,fighter1_rank,fighter1_name,fighter1_result,fighter1_win,finishing_move,fighter2_id,fighter2_rank,fighter2_name,fighter2_result,fighter2_win
0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),0,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),1
1,1983.01,1,4306,Ms1e,Ofuji,1-0 (6-1),1,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),0
2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),1,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),0
3,1983.01,1,4323,J13e,Shiraiwa,0-1 (3-12),0,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),1
4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),0,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),1


In [29]:
results.to_csv('Tournament_Results.csv',index=False)

In [30]:
results.head()

,tournament,day,fighter1_id,fighter1_rank,fighter1_name,fighter1_result,fighter1_win,finishing_move,fighter2_id,fighter2_rank,fighter2_name,fighter2_result,fighter2_win
0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),0,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),1
1,1983.01,1,4306,Ms1e,Ofuji,1-0 (6-1),1,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),0
2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),1,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),0
3,1983.01,1,4323,J13e,Shiraiwa,0-1 (3-12),0,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),1
4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),0,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),1


# Write to Postgress DB

In [6]:
#Read stables file with filled data gaps
stables = pd.read_csv('stables.csv')
tournament_results = pd.read_csv('Tournament_Results.csv')
fighter_img = pd.read_csv('fighter_img.csv')

In [7]:
#Connect to Postgress
rds_connection_string = "biohpcuyvwyfaj:c4482383aab6e82a6695979dcafacc2ca7803503994587b44f4a74b0632c258e@ec2-35-174-35-242.compute-1.amazonaws.com:5432/d7f09hj0q6e97r"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [8]:
engine.table_names()

['tournament_results', 'stables', 'image']

In [11]:
#Insert dataframes into Postgress DB
tournament_results.to_sql(name='tournament_results', con=engine, if_exists='append', index=False)
# stables.to_sql(name='stables', con=engine, if_exists='append', index=False)
# fighter_img.to_sql(name='image', con=engine, if_exists='append', index=False)

KeyboardInterrupt: 

In [ ]:
#Close session
session.close()